In [1]:
from geopy.distance import geodesic
import pandas as pd
import numpy as np
import pickle

## Chargement du polygone de délimitation des berges

In [2]:
pos_eau = []

with open('./data/eau.txt') as f:
    for line in f:
        lon, lat = eval(line)
        pos_eau += [(lat, lon)]

In [3]:
lat_min, lat_max = pos_eau[0][0], pos_eau[0][0]
lon_min, lon_max = pos_eau[0][1], pos_eau[0][1]

for l in pos_eau:
    lat = l[0]
    lon = l[1]
    lat_min, lat_max = min(lat_min, lat), max(lat_max, lat)
    lon_min, lon_max = min(lon_min, lon), max(lon_max, lon)

In [4]:
# Lat, Lon
def distance_to_sea(point1):
    global pos_eau
    
    distance = 100
    
    for point2 in pos_eau:
        distance_tmp = geodesic(point1, point2).km
        distance = min(distance_tmp, distance)
        
    return distance

### Version plus rapide

In [5]:
def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    return np.argmin(dist_2)

In [6]:
# Lat, Lon
def distance_to_sea(point):
    global pos_eau
    
    distance = 100
    
    i = closest_node(point, pos_eau)
        
    return geodesic(point, pos_eau[i]).km

## Création d'une grille des distances

In [7]:
t_x, t_y = 100, 200
tmp_map = pd.DataFrame(columns=['Longitude', 'Latitude', 'Distance'])

for lon in np.linspace(lon_min, lon_max, t_x):
    for lat in np.linspace(lat_min, lat_max, t_y):
        dist = distance_to_sea((lat, lon))
        df2 = pd.DataFrame([[lon, lat, dist]], columns=['Longitude', 'Latitude', 'Distance'])
        
        tmp_map = pd.concat([tmp_map, df2])

## Meilleure précision sur les bords de l'eau

In [8]:
for lat, lon in pos_eau:
    for lon in np.linspace(lon-0.01, lon+0.01, 10):
        for lat in np.linspace(lat-0.01, lat+0.01, 10):
            dist = distance_to_sea((lat, lon))
            df2 = pd.DataFrame([[lon, lat, dist]], columns=['Longitude', 'Latitude', 'Distance'])

            tmp_map = pd.concat([tmp_map, df2])

## Entrainement du modele

In [9]:
from sklearn.neighbors import KNeighborsRegressor

In [10]:
neigh = KNeighborsRegressor(n_neighbors=2, weights='distance')
neigh.fit(tmp_map[['Longitude', 'Latitude']], tmp_map['Distance'])

KNeighborsRegressor(n_neighbors=2, weights='distance')

## Sauvegarde pour utilisation ultérieure

In [11]:
with open('./data/dist_model.plk', 'wb') as file:
    pickle.dump(neigh, file)